<a href="https://colab.research.google.com/github/fysixs/SPICE/blob/master/SPICEOrbitalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[comment]: <> (Written by Andrés Cárdenas)
[comment]: <> (July 2021)
[comment]: <> (www.fysixs.com)
[comment]: <> (andres.cardenas@gmail.com)

<div class="row" align ="left">

 <img src="https://drive.google.com/thumbnail?id=1cJN2J8ByFPCfATK70k6CiHeP0bKGN-Ii" alt="Snow" width="35"> <font face="Gill Sans" color = #667495> &nbsp;**Cool Stuff in Fysixs** 
 <img src="https://drive.google.com/thumbnail?id=1nmgz_xGqeqgvU8wBfJFERu1Zs82bBZ84" alt="Snow" width="55" align ="right">
 <img src="https://drive.google.com/thumbnail?id=1Jytnbufvmu7v3OWJiiXwVLnLnB8ukJPo" alt="Snow" width="90" align ="right">
  <img src="https://i.imgur.com/7c3Iwcl.png" alt="Snow" height="37" align ="right"> <br>
  *Andrés Cárdenas, 2021*, <i><a href="https://www.fysixs.com">www.fysixs.com</a></i>
</font>
</div>



<hr size=5 color=#8D84B5 > </hr> 

<div align="left">
<br>

# <font color = #667495 face="Gill Sans"> &nbsp; &nbsp; &nbsp; **SPICE Orbital Analysis**
## <font color = #667495 face="Gill Sans"> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;*Download, plot and analyze the NAIF data directly* </font>

<br>
</div>

<hr size=5 color=#8D84B5 > </hr> 


<center>
  <img src="https://i.imgur.com/5D0tObE.png" height = 400>
</center>
<div align="right">
  <font align="right" face="Gill Sans"> Data SPICE can give you. <i> Credit: NAIF/JPL</i> <br>
  All NAIF/JPL references at the end </a>. Thank you!
  </font>
</div>



<center>

---
<font size = 4 color=#AB63FA> **⟹** <font size = 4 color=#2E2E2E> ***To get started please run each of the following steps in succession before going on to analysis*** <font size = 4 color=#AB63FA> **⟸** <br>
  Use me <img src="https://i.imgur.com/NVZlUiK.png"> to run a cell</font>

---

In [ ]:
#@title <font size = 4 face="Verdana" color=9467BD> <b>STEP 1: Environment Setup 💻
# Install spiceypy
print("#### INSTALLING SPICEYPY #####")
!pip install spiceypy

# Imports
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import spiceypy as spice
import glob
import os
import sys
import pandas as pd
import ipywidgets as widgets
import datetime
import plotly.io as pio
from bs4 import BeautifulSoup
from IPython.display import clear_output
from ipywidgets import GridBox
from bokeh.io import output_notebook, curdoc
from bokeh.themes import Theme
from bokeh.layouts import column, row, layout
from bokeh.models import CustomJS, Slider, TextInput, Select
from bokeh.models import Range1d, TapTool, Ray, Panel, Tabs, Div
from bokeh.models import PrintfTickFormatter,BasicTickFormatter, HoverTool
from bokeh.plotting import ColumnDataSource, figure, show

output_notebook()

# Plot colors
plt_clr = px.colors.qualitative.Prism

# UI Colors
ui_clr = {
    "button" : "#462770",
    "slider" : "#8B78A6",
    "txt"    : "#91899B",
}

# Bokeh dark theme

theme = Theme(json = {
    "attrs": {
        "Figure" : {
            "background_fill_color": "#20262B",
            "border_fill_color": "#15191C",
            "outline_line_color": "#E0E0E0",
            "outline_line_alpha": 0.25
        },

        "Grid": {
            "grid_line_color": "#E0E0E0",
            "grid_line_alpha": 0.25
        },

        "Axis": {
            # "fixed_location": 0,

            "major_tick_line_alpha": 0.8,
            "major_tick_line_color": "lightgrey",

            "minor_tick_line_alpha": 0.8,
            "minor_tick_line_color": "lightgrey",

            "axis_line_alpha": 0.8,
            "axis_line_color": ui_clr['slider'],

            "major_label_text_color": "lightgrey",
            "major_label_text_font": "Helvetica",
            "major_label_text_font_size": "8pt",

            "axis_label_standoff": 10,
            "axis_label_text_color": "#E0E0E0",
            "axis_label_text_font": "Helvetica",
            "axis_label_text_font_size": "1.25em",
            "axis_label_text_font_style": "normal"
        },

        "Legend": {
            "spacing": 8,
            "glyph_width": 15,

            "label_standoff": 8,
            "label_text_color": "#E0E0E0",
            "label_text_font": "Helvetica",
            "label_text_font_size": "1.025em",

            "border_line_alpha": 0,
            "background_fill_alpha": 0.25,
            "background_fill_color": "#20262B"
        },

        "ColorBar": {
            "title_text_color": "#E0E0E0",
            "title_text_font": "Helvetica",
            "title_text_font_size": "1.025em",
            "title_text_font_style": "normal",

            "major_label_text_color": "#E0E0E0",
            "major_label_text_font": "Helvetica",
            "major_label_text_font_size": "1.025em",

            "background_fill_color": "#15191C",
            "major_tick_line_alpha": 0,
            "bar_line_alpha": 0
        },

        "Title": {
            "text_color": "#E0E0E0",
            "text_font": "Helvetica",
            "text_font_size": "1.15em"
        }
    }
})

curdoc().theme = theme

# Plotly dark theme

pio.templates['my_theme'] = go.layout.Template({
    'layout': {
        'annotationdefaults': {'arrowcolor': '#f2f5fa', 
                               'arrowhead': 0, 'arrowwidth': 1},
        'coloraxis': {'colorbar': {'outlinewidth': 0, 'ticks': ''}},
        'colorscale': {'diverging':   [[0, '#8e0152'], [0.1, '#c51b7d'],
                                      [0.2, '#de77ae'], [0.3, '#f1b6da'], 
                                      [0.4, '#fde0ef'], [0.5, '#f7f7f7'], 
                                      [0.6, '#e6f5d0'], [0.7,'#b8e186'], 
                                      [0.8, '#7fbc41'], [0.9, '#4d9221'],
                                      [1, '#276419']],
                        'sequential': [[0.0, '#0d0887'], [0.1111111111111111,
                                      '#46039f'], [0.2222222222222222, '#7201a8'],
                                      [0.3333333333333333, '#9c179e'],
                                      [0.4444444444444444, '#bd3786'],
                                      [0.5555555555555556, '#d8576b'],
                                      [0.6666666666666666, '#ed7953'],
                                      [0.7777777777777778, '#fb9f3a'],
                                      [0.8888888888888888, '#fdca26'], [1.0,
                                      '#f0f921']],
                   'sequentialminus': [[0.0, '#0d0887'], [0.1111111111111111,
                                       '#46039f'], [0.2222222222222222, '#7201a8'],
                                       [0.3333333333333333, '#9c179e'],
                                       [0.4444444444444444, '#bd3786'],
                                       [0.5555555555555556, '#d8576b'],
                                       [0.6666666666666666, '#ed7953'],
                                       [0.7777777777777778, '#fb9f3a'],
                                       [0.8888888888888888, '#fdca26'], [1.0,
                                       '#f0f921']]},
        'colorway': px.colors.qualitative.Prism,           
    # 'colorway': ["#636efa", "#EF553B", "#00cc96", "#ab63fa", "#FFA15A",
                #  "#19d3f3", "#FF6692", "#B6E880", "#FF97FF", "#FECB52"],
        'font': {'color': '#f2f5fa'},
        'geo': {'bgcolor': 'rgb(17,17,17)',
                'lakecolor': 'rgb(17,17,17)',
                'landcolor': 'rgb(17,17,17)',
                'showlakes': True,
                'showland': True,
                'subunitcolor': '#506784'},
        'hoverlabel': {'align': 'left'},
        'hovermode': 'closest',
        'mapbox': {'style': 'dark'},
        'paper_bgcolor': '#20262B',
        'plot_bgcolor': '#15191C',
        'polar': {'angularaxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''},
                  'bgcolor': 'rgb(17,17,17)',
                  'radialaxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''}},
        'scene': {'xaxis': {'backgroundcolor': '#15191C',
                            'gridcolor': '#506784',
                            'gridwidth': 2,
                            'linecolor': '#506784',
                            'showbackground': False,
                            'showgrid': False,
                            'showline': True,
                            'ticks': 'outside',
                            'zeroline': False,
                            'zerolinecolor': '#C8D4E3'},
                  'yaxis': {'backgroundcolor': '#15191C',
                            'gridcolor': '#506784',
                            'gridwidth': 2,
                            'linecolor': '#506784',
                            'showbackground': False,
                            'showgrid': False,
                            'showline': True,
                            'ticks': 'outside',
                            'zeroline': False,
                            'zerolinecolor': '#C8D4E3'},
                  'zaxis': {'backgroundcolor': '#15191C',
                            'gridcolor': '#506784',
                            'gridwidth': 2,
                            'linecolor': '#506784',
                            'showbackground': False,
                            'showgrid': False,
                            'showline': True,
                            'ticks': 'outside',
                            'zeroline': False,
                            'zerolinecolor': '#C8D4E3'}},           
        'shapedefaults': {'line': {'color': '#f2f5fa'}},
        'sliderdefaults': {'bgcolor': '#C8D4E3', 'bordercolor': 'rgb(17,17,17)', 'borderwidth': 1, 'tickwidth': 0},
        'ternary': {'aaxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''},
                    'baxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''},
                    'bgcolor': 'rgb(17,17,17)',
                    'caxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''}},
        'title': {'x': 0.05},
        'updatemenudefaults': {'bgcolor': '#506784', 'borderwidth': 0},
        'xaxis': {'automargin': True,
                  'gridcolor': '#283442',
                  'linecolor': '#506784',
                  'ticks': '',
                  'title': {'standoff': 15},
                  'zerolinecolor': '#283442',
                  'zerolinewidth': 2},
        'yaxis': {'automargin': True,
                  'gridcolor': '#283442',
                  'linecolor': '#506784',
                  'ticks': '',
                  'title': {'standoff': 15},
                  'zerolinecolor': '#283442',
                  'zerolinewidth': 2} }
})


# Obtain the data source list from NAIF

!wget https://naif.jpl.nasa.gov/naif/data_archived.html
!sed -n '/<!--start data-->/{:a;n;/<!--end data-->/b;p;ba}' data_archived.html > /content/table.html

path = '/content/table.html'
   
# empty list
naif_data = []
   
# for getting the header from
# the HTML file
list_header = []
soup = BeautifulSoup(open(path),'html.parser')
header = soup.find_all("table")[0].find("tr")
  
for items in header:
    try:
        list_header.append(items.get_text())
    except:
        continue
  
# for getting the data 
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
  
for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            if bool(sub_element.a):
              sub_data.append(sub_element.a.get('href'))
            else:
              sub_data.append(sub_element.get_text())
        except:
            continue
    naif_data.append(sub_data)
  
# Storing the data into Pandas
# DataFrame 
spiceDf = pd.DataFrame(data = naif_data, columns = list_header)

# Download SPICE toolkit stuff
!wget https://naif.jpl.nasa.gov/pub/naif/utilities/PC_Linux_64bit/brief
!mv ./brief /content/brief
!chmod u+x /content/brief

# Clone the entire repo.
!git clone -l -s https://github.com/fysixs/SPICE.git /content/SPICE
NAIF_codes = pd.read_csv('/content/SPICE/NAIFCodes.csv')

clear_output()


<hr color=grey width=50% align="left" size=1>
<hr color=grey width=40% align="left" size=1>

In [ ]:
#@title <font size = 4 face="Verdana" color=9467BD> <b>STEP 2: Download Data 🔭{ display-mode: "form" }
#@markdown Please visit <a href="https://naif.jpl.nasa.gov/naif/data_archived.html">NAIF</a> for a detailed description of the mission-data available. <br>
#@markdown The initial and final dates for your selected mission will be displayed.
#@markdown Modify those as needed but beware: the larger the range, the larger the download/wait.
# ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
# Operations
# ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

def download_data(dataSubSet, startDate, stopDate, output):
  with output:
    output.clear_output()
    print("#### GETTING DATA FROM NAIF #####")
    # Prepare the directories
    %cd /content/
    %rm -Rf NAIF
    %mkdir NAIF
    %cd NAIF

    dataURL = ( dataSubSet + "&start=" + startDate.value.isoformat() + 
              "&stop=" + stopDate.value.isoformat() + "&action=Subset" )

    !wget  "$dataURL" -O data.zip
    !unzip data.zip
    !source ./*.tcsh

    subDir = dataSubSet.split("/")[-1]
    %cd /content/NAIF/$subDir

    %cp ../*.tm .
    kernel = glob.glob("./*.tm")[0]

    # %cp ../*.TM .
    # kernel = glob.glob("./*.TM")[0]
    
    print("#### CONNECTING KERNELS #####")
    spice.furnsh(kernel)
  output.clear_output()
  with output:
    %shell /content/brief -a -t /content/NAIF/$subDir/data/spk/*.bsp
  return


def import_data():

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Widgets
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  mission_list = [(name[1], name[0]) for name in enumerate(spiceDf['Mission Name'])]

  mission_select = widgets.Dropdown(
      options=mission_list,
      value=0,
      description='Mission:',
      layout=widgets.Layout(width='auto', height='auto')
  )

  def missionDate(time):
    if time=='start':
      dat = spiceDf['Start Time'][mission_select.value].strip().split('-')
    elif time=='stop':
      dat = spiceDf['Stop Time'][mission_select.value].strip().split('-')
    return datetime.date( int(dat[0]), int(dat[1]), int(dat[2]) )

  startDate = widgets.DatePicker(
      description='Start Date:',
      disabled=False,
      value = missionDate('start'),
      layout=widgets.Layout(width='auto', height='auto')
  )
  
  stopDate = widgets.DatePicker(
      description='Stop Date:',
      disabled=False,
      value = missionDate('stop'),
      layout=widgets.Layout(width='auto', height='auto')
  )

  get_data_button = widgets.Button(description="🛰️ Download data 🛰️",
                                   layout=widgets.Layout(width='auto', height='auto'),
                                   style=widgets.ButtonStyle(button_color='#667495'))
  
  output = widgets.Output()

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Initialize Data
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  data = {} 
  data['mainStartDate'] = startDate 
  data['mainStopDate']  = stopDate

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Interface
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  dataSubSet = spiceDf['Subset Link'][mission_select.value]

  def on_button_clicked(b):
    download_data(dataSubSet, startDate, stopDate, output)
    return
  get_data_button.on_click(on_button_clicked)

  def on_value_change(change):
    startDate.value = missionDate('start')
    stopDate.value  = missionDate('stop')
    return
  mission_select.observe(on_value_change, names='value')

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # UI
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  UI = GridBox(children=[mission_select, startDate, get_data_button, stopDate],
        layout=widgets.Layout(
            width='70%',
            grid_template_columns='300px 300px',
            grid_template_rows='40px 40px',
            grid_gap='10px 20px')
       )

  display( UI )
  display(output)

  return data

data= import_data()


<hr color=grey width=50% align="left" size=1>
<hr color=grey width=40% align="left" size=1>

In [ ]:
#@title <font size = 4 face="Verdana" color=9467BD> <b>STEP 3: Select Objects 🪐 { display-mode: "form" }
#@markdown Select a target, observer and date range of interest. <br> <i> If your selected mission does not contain the data, SPICE will let you know so that you can repeat Step 1 and download data for another mission which covers your target/observer combination during the dates selected. <br>
#@markdown For help on the underlaying SPICE functions please visit: <a href="https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/index.html">SPICE Index</a>. 

def et2datetime(time_et):
  dt = spice.et2utc(time_et, "ISOC", 1, 20).strip().split('T')
  d = dt[0].split('-')
  t = dt[1].split(':')
  return datetime.datetime(int(d[0]), int(d[1]), int(d[2]),
                           int(t[0]), int(t[1]), int(t[2]))
  
def calcKinematics(data, targetnum):

  r = [ data['target'][targetnum]['lt'][i] * spice.clight() for i in range(data['steps']) ]
  v = [ np.linalg.norm([data['target'][targetnum]['vx'][i],
                        data['target'][targetnum]['vy'][i],
                        data['target'][targetnum]['vz'][i]]) for i in range(data['steps']) ]

  data['target'][targetnum]['r'] = r
  data['target'][targetnum]['v'] = v
  data['target'][targetnum]['ω'] = [ v[i]/r[i] for i in range(data['steps']) ]

  return
  
def get_data(data, targetnum):
  # Number of time-points to obtain stored in data['steps']

  # we are going to get positions between these two dates
  utc = [data['plotStartDate'].value.isoformat(),
         data['plotStopDate'].value.isoformat()]

  # Returns seconds since J2000 epoch (January 1, 2000 at 12:00 TerrTime)
  etOne = spice.str2et(utc[0])
  etTwo = spice.str2et(utc[1])
  print("ET Beginning: {}, ET End: {}".format(etOne, etTwo))

  # Creates the array of times requested
  dt = (etTwo-etOne)/data['steps']
  times = [i * dt + etOne for i in range(data['steps'])]

  #Run spkpos to pull location/time data
  #Run spkezr to pull location/vel/time data
  
  #positions, lightTimes = spice.spkpos(orbiter, times, 'J2000', 'NONE',orbiting)

  states, lightTimes = spice.spkezr(data['target'][targetnum]['name'], times,
                                   'J2000', 'NONE', data['observer'])
  
  # Positions is a 3xsteps vector of XYZ positions
  # Light times is a steps vector of time
  data['t'] = {'ET':times, 'UTC': [et2datetime(t) for t in times]}
  data['target'][targetnum]['lt'] = lightTimes
  data['target'][targetnum]['x']  = np.zeros(data['steps'])
  data['target'][targetnum]['y']  = np.zeros(data['steps'])
  data['target'][targetnum]['z']  = np.zeros(data['steps'])
  data['target'][targetnum]['vx'] = np.zeros(data['steps'])
  data['target'][targetnum]['vy'] = np.zeros(data['steps'])
  data['target'][targetnum]['vz'] = np.zeros(data['steps'])
  
  for i in range(data['steps']):
    data['target'][targetnum]['x'][i]  = states[i][0]
    data['target'][targetnum]['y'][i]  = states[i][1]
    data['target'][targetnum]['z'][i]  = states[i][2]
    data['target'][targetnum]['vx'][i] = states[i][3]
    data['target'][targetnum]['vy'][i] = states[i][4]
    data['target'][targetnum]['vz'][i] = states[i][5]

  # Calculate basic kinematics
  calcKinematics(data, targetnum)
  return

def sphere(size, clr): 
  # Spherical domain
  theta = np.linspace(0,2*np.pi,100)
  phi   = np.linspace(0,np.pi,100)
  
  # Set up coordinates for points on the sphere
  x0 = size * np.outer(np.cos(theta),np.sin(phi))
  y0 = size * np.outer(np.sin(theta),np.sin(phi))
  z0 = size * np.outer(np.ones(100),np.cos(phi))
  
  # Set up trace
  trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
  trace.update(showscale=False)

  return trace

def annot_obs():
    strng=dict(showarrow=False, x=0, y=0, z=0,
               xshift=20,
               text=data['observer'], xanchor='left', 
               font=dict(color='antiquewhite',size=12))
    return strng

def annot_target(targetnum):

    initial=dict(showarrow=True,
                 x=data['target'][targetnum]['x'][0],
                 y=data['target'][targetnum]['y'][0],
                 z=data['target'][targetnum]['z'][0],
                 ax=0, ay=-50,
                 arrowhead=3,
                 yshift = 10, 
                 arrowcolor='teal',
                 text=data['target'][targetnum]['name'] + ' Initial',
                 font=dict(color='antiquewhite',size=12))
    
    final=dict(showarrow=True,
                 x=data['target'][targetnum]['x'][-1],
                 y=data['target'][targetnum]['y'][-1],
                 z=data['target'][targetnum]['z'][-1],
                 ax=0, ay=-50, 
                 arrowhead=3,
                 yshift = 10, 
                 arrowcolor='teal',
                 text=data['target'][targetnum]['name'] + ' Final',
                 font=dict(color='antiquewhite',size=12))

    return initial, final

def plot_orbit(data):
  fig = go.Figure()

  targ_x = data['target'][0]['x']
  targ_y = data['target'][0]['y']
  targ_z = data['target'][0]['z']

  # Orbit
  fig.add_trace(go.Scatter3d(x = targ_x, 
                             y = targ_y, 
                             z = targ_z, 
                             mode='lines', # marker_color='orange'
                             ))
  
  # Target
  # Initial
  fig.add_trace(go.Scatter3d(x=[targ_x[0]], y=[targ_y[0]], z=[targ_z[0]], 
                             marker=dict(color='green', size=5)))
  # Final
  fig.add_trace(go.Scatter3d(x=[targ_x[-1]], y=[targ_y[-1]], z=[targ_z[-1]], 
                             marker=dict(color='red', size=5)))

  # Observer
  R_observer = 0.10 * np.sqrt(targ_x[0]**2 + targ_y[0]**2 + targ_z[0]**2)
  fig.add_trace( sphere(R_observer, 'grey') )

  margin=go.layout.Margin(
      l=20, #left margin
      r=20, #right margin
      b=20, #bottom margin
      t=20, #top margin 
  )

  fig.update_layout(template='my_theme', margin=margin,
                    width=600, showlegend=False)
  
  data['orbit_fig_annotations'] = [ annot_obs(), annot_target(0)[0],
                                   annot_target(0)[1] ]
  
  fig.update_layout(scene={'annotations':data['orbit_fig_annotations'],
                           'aspectmode':'data'})

  return fig
  
def analyze_data(data):

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Widgets
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  objectList = [(name[1], name[0]) for name in enumerate(NAIF_codes['NAME'])]

  targetSelect = widgets.Dropdown(
      options=objectList,
      value=0,
      description='Target:',
      layout=widgets.Layout(width='auto', height='auto')
  )

  observerSelect = widgets.Dropdown(
      options=objectList,
      value=0,
      description='Observer:',
      layout=widgets.Layout(width='auto', height='auto')
  )

  dataStartDate = widgets.DatePicker(
      description='Start Date:',
      disabled=False,
      value = data['mainStartDate'].value
  )
  
  dataStopDate = widgets.DatePicker(
      description='Stop Date:',
      disabled=False,
      value = data['mainStopDate'].value
  )

  plot_orbit_button = widgets.Button(description="🚀 Plot Orbit 🚀",
                                   layout=widgets.Layout(width='auto',
                                                         height='auto'),
                                   style=widgets.ButtonStyle(button_color='#667495'))
  
  output = widgets.Output()

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Interface
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  def on_button_clicked(b):
    with output:
      data['target'] = []
      data['target'].append(
          {'name': NAIF_codes['NAME'][targetSelect.value]}
      )
      data['observer']      = NAIF_codes['NAME'][observerSelect.value]
      data['plotStartDate'] = dataStartDate
      data['plotStopDate']  = dataStopDate

      get_data(data, 0)
      
      data['orbit_fig'] = plot_orbit(data)

      output.clear_output()
      data['orbit_fig'].show()
    return
  plot_orbit_button.on_click(on_button_clicked)

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # UI
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ  

  UI = GridBox(children=[targetSelect, dataStartDate,
                         observerSelect, dataStopDate,
                         plot_orbit_button],
        layout=widgets.Layout(
            width='70%',
            grid_template_columns='300px 300px',
            grid_template_rows='40px 40px 40px',
            grid_gap='10px 20px')
       )

  display( UI )
  display( output )

  return

data['steps']    = 8000
data['observer'] = ""
data['target']   = []

analyze_data(data)
 

<hr color=grey width=50% align="left" size=1>
<hr color=grey width=40% align="left" size=1>

In [ ]:
#@title <font size = 4 face="Verdana" color=9467BD> <b>STEP 4: Add more Targets (Optional) 🪐{ display-mode: "form" }
#@markdown Stuff about the followoing
def add_target_plot(data, targetnum):
  targ_x = data['target'][targetnum]['x']
  targ_y = data['target'][targetnum]['y']
  targ_z = data['target'][targetnum]['z']
  fig    = data['orbit_fig']

  # Orbit
  fig.add_trace(go.Scatter3d(x = targ_x, 
                             y = targ_y, 
                             z = targ_z, 
                             mode='lines', # marker_color='orange'
                             ))
  
  # Target
  # Initial
  fig.add_trace(go.Scatter3d(x=[targ_x[0]], y=[targ_y[0]], z=[targ_z[0]], 
                             marker=dict(color='green', size=5)))
  # Final
  fig.add_trace(go.Scatter3d(x=[targ_x[-1]], y=[targ_y[-1]], z=[targ_z[-1]], 
                             marker=dict(color='red', size=5)))
  
  data['orbit_fig_annotations'].append(annot_target(targetnum)[0])
  data['orbit_fig_annotations'].append(annot_target(targetnum)[1])
  
  fig.update_layout(scene={'annotations':data['orbit_fig_annotations']})

  return fig
  
def add_target(data):

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Widgets
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  objectList = [(name[1], name[0]) for name in enumerate(NAIF_codes['NAME'])]

  targetSelect = widgets.Dropdown(
      options=objectList,
      value=0,
      description='Target:',
      layout=widgets.Layout(width='auto', height='auto')
  )

  plot_orbit_button = widgets.Button(description="🚀 Add Orbit 🚀",
                                   layout=widgets.Layout(width='auto',
                                                         height='auto'),
                                   style=widgets.ButtonStyle(button_color='#667495'))
  
  output = widgets.Output()

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Interface
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  def on_button_clicked(b):
    with output:
      data['target'].append(
          {'name': NAIF_codes['NAME'][targetSelect.value]}
      )
      targetnum = len(data['target']) - 1
      get_data(data, targetnum)
      
      data['orbit_fig'] = add_target_plot(data, targetnum)

      output.clear_output()
      data['orbit_fig'].show()
    return
  plot_orbit_button.on_click(on_button_clicked)

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # UI
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ  

  UI = widgets.HBox([targetSelect, plot_orbit_button])

  display( UI )
  display( output )

  return

add_target(data)
 

<hr size=3 color=#8D84B5 > </hr> 
<br>

#<font color = #667495 face="Gill Sans"> **Graphical Analysis**</font>


The Orbit Component Graph shows the x,y,z position of the orbiter.

In [ ]:
#@title <font color=#667495> ***Kinematic Plots*** { display-mode: "form" }

def plot_mags(kin_var, labels, cds):

  p = figure(plot_width=600, plot_height=400,
             title = labels[0],
             tools = "box_zoom, reset, hover", toolbar_location = 'above',
             x_axis_type = 'datetime' )
  
  p.line(x='t', y=kin_var, source=cds,
         line_color=plt_clr[3], line_width=2, line_alpha=0.9)
  
  p.xaxis.axis_label = 'time'
  p.yaxis.axis_label = labels[1]
  p.axis.axis_label_text_font_size = '9pt'

  p.yaxis.formatter = BasicTickFormatter(power_limit_high=3, precision=1)

  TOOLTIPS = [("date", '@t{%F}'),
              (labels[1], f'@{kin_var}'+'{%0.3e}'),
  ]
  hover = p.select(dict(type=HoverTool))
  hover.tooltips = TOOLTIPS
  hover.formatters = {f'@{kin_var}': 'printf', '@t': 'datetime'}

  return p

def plot_comps(kin_var, labels, cds):

  p = figure(plot_width=600, plot_height=400,
             title = labels[0],
             tools = "box_zoom, reset, hover", toolbar_location = 'above',
             x_axis_type = 'datetime' )
  
  p.line(x='t', y=f'{kin_var}x', source=cds,
         line_color=plt_clr[0], line_width=2, line_alpha=0.9,
         legend_label = f'{kin_var}x' )
  
  p.line(x='t', y=f'{kin_var}y', source=cds,
         line_color=plt_clr[1], line_width=2, line_alpha=0.9,
         legend_label = f'{kin_var}y' )
  
  p.line(x='t', y=f'{kin_var}z', source=cds,
         line_color=plt_clr[2], line_width=2, line_alpha=0.9,
         legend_label = f'{kin_var}z' )
  
  p.xaxis.axis_label = 'time'
  p.yaxis.axis_label = labels[1]
  p.axis.axis_label_text_font_size = '9pt'
  p.legend.click_policy="hide"

  p.yaxis.formatter = BasicTickFormatter(power_limit_high=3, precision=1)

  TOOLTIPS = [("date", '@t{%F}'),
              (labels[1]+'x', f'@{kin_var}x'+'{%0.3e}'),
              (labels[1]+'y', f'@{kin_var}y'+'{%0.3e}'),
              (labels[1]+'z', f'@{kin_var}z'+'{%0.3e}')              
  ]
  hover = p.select(dict(type=HoverTool))
  hover.tooltips = TOOLTIPS
  hover.formatters = {f'@{kin_var}x': 'printf',f'@{kin_var}y': 'printf',f'@{kin_var}z': 'printf', '@t': 'datetime'}

  return p

def kinplots(data):
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Widgets
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  targetsN   = len(data['target'])
  targetList = [ data['target'][i]['name'] for i in range(targetsN) ]

  target_selection = Select(title="Target:", 
                         value=targetList[0], 
                         options=targetList)
  target_selection.width = 200

  observer = Div(text=f"""Observer: {data['observer']}""", width=200)

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Prepare Data
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  targetnum = targetList.index(target_selection.value)

  cds = ColumnDataSource(data= {'t' : data['t']['UTC'],
                                'rx': data['target'][targetnum]['x'],
                                'ry': data['target'][targetnum]['y'],
                                'rz': data['target'][targetnum]['z'],
                                'r' : data['target'][targetnum]['r'],
                                'vx': data['target'][targetnum]['vx'],
                                'vy': data['target'][targetnum]['vy'],
                                'vz': data['target'][targetnum]['vz'],
                                'v' : data['target'][targetnum]['v']
                                })
  
  cds_all = ColumnDataSource(data= {'target': data['target']})

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Interface
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  redraw_code = """
  // Get new selection
  var target_list = target.options
  var target_idx  = target_list.indexOf(target.value)

  // Start with fresh data
  cds.data['rx'] = []
  cds.data['ry'] = []
  cds.data['rz'] = []
  cds.data['r'] = []
  cds.data['vx'] = []
  cds.data['vy'] = []
  cds.data['vz'] = []
  cds.data['v'] = []
  
  // Load new selection
  var N = cds.data['t'].length
  for (var i = 0; i < N; i++) {
    cds.data['rx'].push( cds_all.data['target'][target_idx]['x'][i] )
    cds.data['ry'].push( cds_all.data['target'][target_idx]['y'][i] )
    cds.data['rz'].push( cds_all.data['target'][target_idx]['z'][i] )
    cds.data['r'].push( Math.sqrt(cds_all.data['target'][target_idx]['x'][i]*cds_all.data['target'][target_idx]['x'][i] +
                                  cds_all.data['target'][target_idx]['y'][i]*cds_all.data['target'][target_idx]['y'][i] +
                                  cds_all.data['target'][target_idx]['z'][i]*cds_all.data['target'][target_idx]['z'][i] ) )
    cds.data['vx'].push( cds_all.data['target'][target_idx]['vx'][i] )
    cds.data['vy'].push( cds_all.data['target'][target_idx]['vy'][i] )
    cds.data['vz'].push( cds_all.data['target'][target_idx]['vz'][i] )
    cds.data['v'].push( Math.sqrt(cds_all.data['target'][target_idx]['vx'][i]*cds_all.data['target'][target_idx]['vx'][i] +
                                  cds_all.data['target'][target_idx]['vy'][i]*cds_all.data['target'][target_idx]['vy'][i] +
                                  cds_all.data['target'][target_idx]['vz'][i]*cds_all.data['target'][target_idx]['vz'][i] ) )
  }
  cds.change.emit()
  """

  redraw_callback = CustomJS(
      args = {'cds' : cds, 'cds_all' : cds_all, 'target' : target_selection},
      code = redraw_code
      )

  target_selection.js_on_change('value', redraw_callback)

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # UI
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  labels  = [ "Orbital Position", "position (km)" ]
  posplot = plot_comps('r', labels, cds)
  labels  = [ "Orbital Distance", "distance (km)" ]
  distplot = plot_mags('r', labels, cds)

  labels  = [ "Orbital Velocity", "velocity (km/s)" ]
  velplot = plot_comps('v', labels, cds)
  labels  = [ "Orbital Speed", "speed (km/s)" ]
  speedplot = plot_mags('v', labels, cds)

  tab1 = Panel(child=row([posplot, distplot]), title="Position")
  tab2 = Panel(child=row([velplot, speedplot]), title="Velocity")

  plots = Tabs(tabs=[tab1, tab2], tabs_location="left")  
  UI = column( row([ target_selection, observer]), plots )

  show(UI)
  return

kinplots(data)


<hr size=3 color=#8D84B5 align="left"> </hr> 


# <font color = #667495 face="Gill Sans"> *References*</font>
<hr size=2 color=grey width=30% align="left"> </hr> 

1. Acton, C.H.; "Ancillary Data Services of NASA's Navigation and Ancillary Information Facility;" Planetary and Space Science, Vol. 44, No. 1, pp. 65-70, 1996.

2. Charles Acton, Nathaniel Bachman, Boris Semenov, Edward Wright; A look toward the future in the handling of space science mission geometry; Planetary and Space Science (2017); 
DOI 10.1016/j.pss.2017.02.013
https://doi.org/10.1016/j.pss.2017.02.013
